## Newton's Method

Wan Dang - 02/10/2025

### Imported Packages

In [53]:
import numpy as np 
import math
import sympy as sym
import mpmath

In [54]:
def epsilon_machine():
    return np.finfo(np.float64).eps

Consider the equations:

(A) $f(x)=(400x^6 + (5x^2-1)^4 + 60x^2 +8)e - 24e^{5x^2}$

(B) $f(x)=(500x^6 + (5x^2-1)^4 + 60x^2 +8)e - 24e^{5x^2}$

(C) $f(x)=(600x^6 + (5x^2-1)^4 + 60x^2 +8)e - 24e^{5x^2}$

Each version has one positive solution, which is between 0 and 1, that we are looking for. In this project, we will attempt an approach, which will fail to get 6 correct digits for part (B), but will get there faster. For each version, you will find the solution using Newton's method, and study the convergence to the solution.


In [55]:
# Define a Newton's method which prints out each iteration
def newton_method_iteration(f, df, x0, tol, max_iter, r):
    '''
    Implements Newton's Method for finding the root of a function f(x) with step-by-step iteration details.

    Parameters:
    -----------
    f : sympy expression
        The function for which the root is being approximated.
    df : sympy expression
        The derivative of the function f.
    x0 : float
        The initial guess for the root.
    tol : float
        The tolerance value to determine convergence.
    max_iter : int
        The maximum number of iterations allowed.
    r : float
        The actual root (if known) for error analysis.

    Prints:
    -------
    Iteration details in a tabular format with the following columns:
        - i: Iteration number
        - xi: Current approximation of the root
        - e_i: Absolute error |xi - r|
        - e_i / (e_(i-1)^2): Ratio used to observe quadratic convergence
        - e_i / e_(i-1): Ratio used to observe linear convergence

    Returns:
    --------
    tuple:
        (x_final, iterations)
        - x_final (float): The computed approximation of the root.
        - iterations (int): The number of iterations performed.

    Notes:
    ------
    - The function prints the progress of each iteration, showing how the approximation improves.
    - If the error falls below the tolerance `tol`, the function returns early.
    - If the maximum number of iterations is reached before convergence, the function returns the last computed approximation.
    '''
    # Column Headers
    header = f"{'i':<4} | {'xi':<25} | {'e_i = |xi - r|':<25} | {'e_i / (e_(i-1)^2)':<25} | {'e_i / e_(i-1)':<25}"
    separator = "-" * len(header)

    print(header)
    print(separator)
    # Calculate the absolute error between the guess and the root
    e_i = abs(x0 - r)
    print(f"{0:<4} | {x0:<25.16f} | {e_i:<25.16f}")
    # For loop for each iterations
    for i in range(1, max_iter + 1):
        # Assign x0 to the function fx and dfx
        fx = f.subs(x, x0)
        dfx = df.subs(x, x0)
        # Calculate the new x and absoulte error
        x_i = x0 - fx / dfx
        e_next = abs(x_i - r)
        e_square = e_next / e_i**2
        # Print the iteration
        # Handling small numbers to prevent large ratios
        e_i_squared = e_i**2 if e_i > epsilon_machine() else epsilon_machine()
        ratio_squared = e_next / e_i_squared
        ratio_linear = e_next / e_i if e_i > epsilon_machine() else 0

        print(f"{i:<4} | {x_i:<25.16f} | {e_next:<25.16f} | {ratio_squared:<25.16f} | {ratio_linear:<25.16f}")
        # Check if the absolute error less than the tolerance
        if e_next < tol:
            return x_i, i
            
        e_i = e_next  # Update previous error
        x0 = x_i  # Update current x value
    return x0, i

### Test case 1: $f(x)=(400x^6 + (5x^2-1)^4 + 60x^2 +8)e - 24e^{5x^2}$

In [56]:
x = sym.symbols('x')
f = (400*x**6 + (5*x**2-1)**4 + 60*x**2 + 8)*math.e - 24*math.e**(5*x**2)
# Calculate derivative of f (d/df)
df = sym.diff(f)
r = 0.2528577325758161656455234211326244910052 # Root getting from Wolfram

In [57]:
root, iteration = newton_method_iteration(f, df, 1, epsilon_machine(), 50, r)

i    | xi                        | e_i = |xi - r|            | e_i / (e_(i-1)^2)         | e_i / e_(i-1)            
--------------------------------------------------------------------------------------------------------------------
0    | 1.0000000000000000        | 0.7471422674241839       
1    | 0.9269207318782170        | 0.6740629993024000        | 1.2075187313042300        | 0.9021882828638130       
2    | 0.8528279173811500        | 0.5999701848053340        | 1.3204704654712500        | 0.8900802824457860       
3    | 0.7752512260673570        | 0.5223934934915400        | 1.4512372636313800        | 0.8706990892573030       
4    | 0.6902358550236400        | 0.4373781224478240        | 1.6027343237653300        | 0.8372579825305700       
5    | 0.5957260576168810        | 0.3428683250410640        | 1.7923105203431300        | 0.7839174102311580       
6    | 0.5003657697976590        | 0.2475080372218430        | 2.1053997859822800        | 0.7218748981615600       
7  

The penultimate rows have a large value for $\frac{e_{i+1}}{e_{i}^2}$ ($\approx 124667117712678$). This suggests numerical precision errors as Newton's method gets close to the root, and this is expected because floating-point arithmetic cannot handle extremely small values perfectly. So we will skip the last two rows .

We can see $\frac{e_{i+1}}{e_{i}^2}$ converges to $\approx 3.33354253491026$. Compared with the rate of convergence (ROC) we computed is:
\begin{equation*}
    ROC = \frac{e_{i+1}}{e_{i}^2} \approx \frac{\mid{f''(r)}\mid}{2\mid{f'(r)}\mid} = 3.32844045038455
\end{equation*}
The ratio and the theoretical value should agree to at least 1 decimal place.

### Test case 2: $f(x)=(500x^6 + (5x^2-1)^4 + 60x^2 +8)e - 24e^{5x^2}$


In [58]:
x = sym.symbols('x')
f = (500*x**6 + (5*x**2-1)**4 + 60*x**2 + 8)*math.e - 24*math.e**(5*x**2)
df = sym.diff(f, x)
r = 1/math.sqrt(5)

In [59]:
root, iteration = newton_method_iteration(f, df, 1, epsilon_machine(), 100, r)

i    | xi                        | e_i = |xi - r|            | e_i / (e_(i-1)^2)         | e_i / e_(i-1)            
--------------------------------------------------------------------------------------------------------------------
0    | 1.0000000000000000        | 0.5527864045000421       
1    | 0.9344848209841330        | 0.4872712254841750        | 1.5946157875717300        | 0.8814819277707800       
2    | 0.8729843030195000        | 0.4257707075195420        | 1.7932227797401300        | 0.8737858614501140       
3    | 0.8161658285168350        | 0.3689522330168770        | 2.0352538489552700        | 0.8665514712515590       
4    | 0.7644272376606050        | 0.3172136421606470        | 2.3302985774727300        | 0.8597688637546140       
5    | 0.7179265842134520        | 0.2707129887134950        | 2.6903289236350900        | 0.8534090364764220       
6    | 0.6766279980863480        | 0.2294144025863900        | 3.1304191539788600        | 0.8474451250995850       
7  

Doing calculation, we can easily see that $f(r)=f'(r)=f''(r)=f^{(3)}(r)=f^{(4)}(r)=0$, and $f^{(5)}(r)\neq0\Rightarrow$ multiplicity for $r=\frac{1}{\sqrt5}$ is $m=5$.

$\Rightarrow$ Newton's method is locally and linearly convergent, and $\lim_{i\to\infty}\frac{e_{i+1}}{e_i}=\frac{m-1}{m}=\frac{4}{5}=0.8$

From the Newton iteration, we can see that $\frac{e_{i+1}}{e_i}$ converges to nearly 0.8.


### Test case 3: $f(x)=(600x^6 + (5x^2-1)^4 + 60x^2 +8)e - 24e^{5x^2}$

In [60]:
x = sym.symbols('x')
f = (600*x**6 + (5*x**2-1)**4 + 60*x**2 + 8)*math.e - 24*math.e**(5*x**2)
df = sym.diff(f, x)
r = 0.8432553110233654628363872594116270343097

In [61]:
root, iteration = newton_method_iteration(f, df, 1, epsilon_machine(), 100, r)

i    | xi                        | e_i = |xi - r|            | e_i / (e_(i-1)^2)         | e_i / e_(i-1)            
--------------------------------------------------------------------------------------------------------------------
0    | 1.0000000000000000        | 0.1567446889766345       
1    | 0.9433791412651900        | 0.1001238302418250        | 4.0752268249160700        | 0.6387701611807070       
2    | 0.8966229560464130        | 0.0533676450230470        | 5.3235719551259000        | 0.5330164147151640       
3    | 0.8636148336657520        | 0.0203595226423868        | 7.1484439006926500        | 0.3814956165593310       
4    | 0.8471721293118230        | 0.0039168182884571        | 9.4492696329189300        | 0.1923826190454310       
5    | 0.8434280691305090        | 0.0001727581071440        | 11.2608607756570000       | 0.0441067454298631       
6    | 0.8432556624696490        | 0.0000003514462834        | 11.7755738649232000       | 0.0020343258514386       
7  

Similar to the test case (1), we will drop the last row. We can see $\frac{e_{i+1}}{e_{i}^2}$ converges to $\approx 11.8011401544340$. Compared with the rate of convergence (ROC) we computed is:
\begin{equation*}
    ROC = \frac{e_{i+1}}{e_{i}^2} \approx \frac{\mid{f''(r)}\mid}{2\mid{f'(r)}\mid} = 11.8004676963633
\end{equation*}
The ratio and the theoretical value in this case,  agree to 2 decimal place.

## Conclusion 

Newton’s method exhibits different convergence behaviors depending on the function's characteristics. In cases where the root is simple, such as in Case A and Case C, the method converges quadratically, with Case C achieving the fastest convergence due to a higher rate of convergence constant ($ C \approx 11.80 $). However, in Case B, the root has a multiplicity of 5, leading to a loss of quadratic convergence and a much slower linear convergence rate ($e_{i+1} / e_i \approx 0.8$), requiring significantly more iterations (100 vs. 6-15 in the other cases). Additionally, numerical precision issues arise in the final iterations, particularly when error values approach machine precision ($\approx 10^{-16}$). These results highlight the importance of considering function properties, such as derivative behavior and root multiplicity, when applying Newton’s method for efficient and accurate root-finding.